# 幂剩余

In [1]:
def modexp(x, n, p): #计算x的n次幂除以p的余数
    if n == 0:
        return 1
    t = (x * x) % p
    tmp = modexp(t, n//2, p)
    if n%2 != 0:
        tmp = (tmp * x) % p
    return tmp

In [6]:
 modexp(23, 1, 7)

2

# 最大公因数与逆元

In [8]:
def ext_gcd(x, y):
    if y == 0:
        return(x, 1, 0) #x=1*x+0*y
    else:
        (d, a, b) = ext_gcd(y, x%y) #d=gcd(x,y)=ax+by=gcd(y, x%y)=ay+b(x%y)=bx+[a-(x//y)*b]y
        return(d, b, a-(x//y)*b) #返回：(最大公因数,x的系数,y的系数)（若d=1，则y的系数是y关于modx的一个逆元）

In [9]:
ext_gcd(25, 9)

(1, 4, -11)

# RSA算法

In [10]:
def RSAgenKeys(p, q):
    n = p * q
    pqminus = (p-1) * (q-1)
    e = int(random.random() * n) #random.random() 返回一个0到1的随机数
    while gcd(pqminus, e) != 1:
        e = int(random.random() * n) #随机到一个与pqminus互质的e
        d, a, b = ext_gcd(pqminus, e) #成一个e(mod pqminus)的逆元d（[a*pqminus+b*e](mod pqminus)=1）
    '''保证d大于0'''
    if b < 0:
        d = pqminus + b
    else:
        d = b
    return((e, d, n))
    
def RSAencrypt(m,e,n): #加密
    '''bit_length()返回n用二进制表示的位数，再整除8得到m的16进制表示的位数上限，
    因为每个字符由两个数表示，因此字节数还要×2。得到字符串m分割后每一块的位数。'''
    chunks = toChunks(m, n.bit_length() // 8 * 2)
    encList = []
    for messChunk in chunks:
        c = modexp(messChunk, e, n) #模指数运算：计算messChunk^e(mod n)
        encList.append(c)
    return encList
    
def RSAdecrypt(chunkList, d, n): #解密
    rList = []
    for c in chunkList:
        m = modexp(c,d,n)
        rList.append(m)
    return chunksToPlain(rList, n.bit_length() // 8 * 2)

In [11]:
'''将字符串转换为数字块列表'''
def toChunks(m, chunkSize):
    byteMess = bytes(m, 'utf-8') #用utf-8的编码返回m的编码
    hexString = '' #hex表示16进制
    for b in byteMess:
        hexString = hexString + ("%02x" % b) #表示用16进制输出每一个字符，且每个字符都是两位（这样可以自动补0）

    numChunks = len(hexString) // chunkSize #计算块数
    chunkList = []
    for i in range(0, numChunks*chunkSize+1, chunkSize): #将完整的字符串切割成numChunks块小字符串
        chunkList.append(hexString[i:i+chunkSize])
    chunkList = [eval('0x'+x) for x in chunkList if x] #'0x'表示16进制转10进制，将列表中的字符全部转换为10进制的数
    return chunkList
    
def chunksToPlain(clist, chunkSize):
    hexList = []
    for c in clist:
        hexString = hex(c)[2:] #将c转换为16进制字符串，[2:]意思是从第三位开始表示，是因为16进制字符串前两位是指示符0x
        clen = len(hexString)
        hexList.append('0' * ((chunkSize - clen) % 2) + hexString) #补0
    
    hstring = "".join(hexList)
    messArray = bytearray.fromhex(hstring)
    return messArray.decode('utf-8')